In [ ]:
"""
activity i need to capture:

- total commits - 
- total lines of code added - 
- total lines of code removed - 

- commits per repository -
    - display org and reponame together with number of commits
- repositories created - 
    - name of repo
- issues opened 
    - name of repo and headline of issue
- pull requests opened
    - name of repo and title of pull request


# how it works

- github cronjob daily at the end of my day 00:00 - collects data until back 00:01
- telegram bot sends me a message with the data formatted
"""

In [30]:
import requests
from datetime import datetime, timedelta
import pytz
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def fetch_last_event_local_time(username, token):
    if not username or not token:
        raise ValueError("GitHub username or token is not set.")
    
    headers = {'Authorization': f'token {token}'}
    url = f'https://api.github.com/users/{username}/events'
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching events: {response.status_code}")

    events = response.json()
    if not events:
        print("No events found for the user.")
        return None
    
    last_event_time_utc = events[0]['created_at']  # The last event's timestamp

    # Convert to local time
    dt = datetime.strptime(last_event_time_utc, '%Y-%m-%dT%H:%M:%SZ')
    utc_time = dt.replace(tzinfo=pytz.UTC)
    local_time = utc_time.astimezone()  # Convert to local timezone

    return local_time

In [31]:
def get_time_24_hours_back(event_time):
    # Get the current time in the same timezone as the event time
    current_time = datetime.now(event_time.tzinfo)
    
    # Calculate the time 24 hours back
    time_24_hours_back = current_time - timedelta(hours=24)
    
    return time_24_hours_back

Current time: 2024-09-07 13:09:56.142089+02:00
Time 24 hours back: 2024-09-06 13:09:56.142084+02:00


In [54]:
def fetch_user_events(username, token):
    if not username or not token:
        raise ValueError("GitHub username or token is not set.")
    
    headers = {
        'Accept': 'application/vnd.github+json',
        'Authorization': f'Bearer {token}',
        'X-GitHub-Api-Version': '2022-11-28'
    }
    url = f'https://api.github.com/users/{username}/events'
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching events: {response.status_code}")

    events = response.json()
    print(events)
    return events

def filter_commits_last_24_hours(events, since, until):
    commits = []
    for event in events:
        if event['type'] == 'PushEvent':
            event_time = datetime.strptime(event['created_at'], '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.UTC)
            if since <= event_time <= until:
                for commit in event['payload']['commits']:
                    repo_name = event['repo']['name']
                    commit_sha = commit['sha']
                    commits.append((repo_name, commit_sha))
    return commits

In [55]:
#for each commit get commit details 
"""
{'id': '41714209607', 'type': 'PushEvent', 'actor': {'id': 68147541, 'login': 'yachty66', 'display_login': 'yachty66', 'gravatar_id': '', 'url': 'https://api.github.com/users/yachty66', 'avatar_url': 'https://avatars.githubusercontent.com/u/68147541?'}, 'repo': {'id': 850812657, 'name': 'yachty66/ai_code_reviewer', 'url': 'https://api.github.com/repos/yachty66/ai_code_reviewer'}, 'payload': {'repository_id': 850812657, 'push_id': 20126201325, 'size': 1, 'distinct_size': 1, 'ref': 'refs/heads/main', 'head': '2f215a123fd93134818ffae7377ee79d523a805e', 'before': '1af1355599060a088cf9359ce942a5abf5fd2b1d', 'commits': [{'sha': '2f215a123fd93134818ffae7377ee79d523a805e', 'author': {'email': '68147541+yachty66@users.noreply.github.com', 'name': 'Max Hager'}, 'message': 'test', 'distinct': True, 'url': 'https://api.github.com/repos/yachty66/ai_code_reviewer/commits/2f215a123fd93134818ffae7377ee79d523a805e'}]}, 'public': True, 'created_at': '2024-09-07T11:42:44Z'}
"""
def fetch_commit_details(username, repo, sha, token):
    headers = {'Authorization': f'token {token}'}
    url = f'https://api.github.com/repos/{repo}/commits/{sha}'
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching commit details for {sha}: {response.status_code}")
    
    return response.json()

def calculate_lines_added(commits, token):
    total_lines_added = 0
    for repo_name, commit_sha in commits:
        commit_details = fetch_commit_details(repo_name.split('/')[0], repo_name, commit_sha, token)
        total_lines_added += commit_details['stats']['additions']
    return total_lines_added

def calculate_lines_removed(commits, token):
    total_lines_removed = 0
    for repo_name, commit_sha in commits:
        commit_details = fetch_commit_details(repo_name.split('/')[0], repo_name, commit_sha, token)
        total_lines_removed += commit_details['stats']['deletions']
    return total_lines_removed

def count_commits_per_repo(commits):
    repo_commit_count = {}
    for repo_name, _ in commits:
        if repo_name not in repo_commit_count:
            repo_commit_count[repo_name] = 0
        repo_commit_count[repo_name] += 1
    return [{'repo_name': repo, 'commit_count': count} for repo, count in repo_commit_count.items()]

def filter_repos_created_last_24_hours(events, since, until):
    repos_created = []
    for event in events:
        if event['type'] == 'CreateEvent' and event['payload']['ref_type'] == 'repository':
            event_time = datetime.strptime(event['created_at'], '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.UTC)
            if since <= event_time <= until:
                repos_created.append(event['repo']['name'])
    return repos_created

def filter_repos_created_last_24_hours(events, since, until):
    repos_created = []
    for event in events:
        if event['type'] == 'CreateEvent' and event['payload']['ref_type'] == 'repository':
            event_time = datetime.strptime(event['created_at'], '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.UTC)
            if since <= event_time <= until:
                repos_created.append(event['repo']['name'])
    return repos_created

def filter_issues_opened_last_24_hours(events, since, until):
    issues_opened = []
    for event in events:
        if event['type'] == 'IssuesEvent' and event['payload']['action'] == 'opened':
            event_time = datetime.strptime(event['created_at'], '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.UTC)
            if since <= event_time <= until:
                repo_name = event['repo']['name']
                issue_title = event['payload']['issue']['title']
                issues_opened.append({'repo_name': repo_name, 'issue_title': issue_title})
    return issues_opened

def filter_pull_requests_opened_last_24_hours(events, since, until):
    pull_requests_opened = []
    for event in events:
        if event['type'] == 'PullRequestEvent' and event['payload']['action'] == 'opened':
            event_time = datetime.strptime(event['created_at'], '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.UTC)
            if since <= event_time <= until:
                repo_name = event['repo']['name']
                pr_title = event['payload']['pull_request']['title']
                pull_requests_opened.append({'repo_name': repo_name, 'pr_title': pr_title})
    return pull_requests_opened

def filter_discussions_started_last_24_hours(events, since, until):
    discussions_started = []
    for event in events:
        if event['type'] == 'DiscussionEvent' and event['payload']['action'] == 'created':
            event_time = datetime.strptime(event['created_at'], '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.UTC)
            if since <= event_time <= until:
                repo_name = event['repo']['name']
                discussion_title = event['payload']['discussion']['title']
                discussions_started.append({'repo_name': repo_name, 'discussion_title': discussion_title})
    return discussions_started

# Example usage
username = os.getenv('GITHUB_USER')
token = os.getenv('GITHUB_TOKEN')

# Step 1: Get the last event time in local timezone
event_time = fetch_last_event_local_time(username, token)
if event_time:
    # Step 2: Calculate the time 24 hours back
    time_24_hours_back = get_time_24_hours_back(event_time)
    
    # Step 3: Fetch all recent events
    events = fetch_user_events(username, token)
    
    if events:
        # Step 4: Filter commits from the last 24 hours
        commits_last_24_hours = filter_commits_last_24_hours(events, time_24_hours_back, event_time)
        
        # Step 5: Calculate total lines of code added
        total_lines_added = calculate_lines_added(commits_last_24_hours, token)
        print("Total lines of code added in the last 24 hours:", total_lines_added)
        
        # Step 6: Calculate total lines of code removed
        total_lines_removed = calculate_lines_removed(commits_last_24_hours, token)
        print("Total lines of code removed in the last 24 hours:", total_lines_removed)
        
        # Step 7: Count the number of commits
        total_commits = len(commits_last_24_hours)
        print("Total commits in the last 24 hours:", total_commits)
        
        # Step 8: Count commits per repository
        commits_per_repo = count_commits_per_repo(commits_last_24_hours)
        print("Commits per repository in the last 24 hours:", commits_per_repo)
        
        # Step 9: Filter repositories created in the last 24 hours
        repos_created_last_24_hours = filter_repos_created_last_24_hours(events, time_24_hours_back, event_time)
        print("Repositories created in the last 24 hours:", repos_created_last_24_hours)
        
        # Step 10: Filter issues opened in the last 24 hours
        issues_opened_last_24_hours = filter_issues_opened_last_24_hours(events, time_24_hours_back, event_time)
        print("Issues opened in the last 24 hours:", issues_opened_last_24_hours)
        
        # Step 11: Filter pull requests opened in the last 24 hours
        pull_requests_opened_last_24_hours = filter_pull_requests_opened_last_24_hours(events, time_24_hours_back, event_time)
        print("Pull requests opened in the last 24 hours:", pull_requests_opened_last_24_hours)
        
        # Step 12: Filter discussions started in the last 24 hours
        discussions_started_last_24_hours = filter_discussions_started_last_24_hours(events, time_24_hours_back, event_time)
        print("Discussions started in the last 24 hours:", discussions_started_last_24_hours)

[{'id': '41714546870', 'type': 'PushEvent', 'actor': {'id': 68147541, 'login': 'yachty66', 'display_login': 'yachty66', 'gravatar_id': '', 'url': 'https://api.github.com/users/yachty66', 'avatar_url': 'https://avatars.githubusercontent.com/u/68147541?'}, 'repo': {'id': 850812657, 'name': 'yachty66/ai_code_reviewer', 'url': 'https://api.github.com/repos/yachty66/ai_code_reviewer'}, 'payload': {'repository_id': 850812657, 'push_id': 20126432558, 'size': 1, 'distinct_size': 1, 'ref': 'refs/heads/main', 'head': 'd1d1160d4f89e7a873b36ecd80528a6924344205', 'before': '79adbc5fc85d5723cc5fbb44add1ec27fee7dc95', 'commits': [{'sha': 'd1d1160d4f89e7a873b36ecd80528a6924344205', 'author': {'email': '68147541+yachty66@users.noreply.github.com', 'name': 'Max Hager'}, 'message': 'Update execution count and statistics in main.ipynb: adjusted lines of code added/removed, updated commit counts per repository, and added new pull request information.', 'distinct': True, 'url': 'https://api.github.com/repos